## [Home](file:///C:/SQLScriptsLibrary/Home.ipynb)

## Auditing

> - Some Stuff (Query 1)
>     - More Stuff (Query 2)
>     - Some Info
>         - More Info- (Query 3.1)
>         - Even More Info IP - (Query 3.2)

In [27]:
/* Database audit specifications in a SQL Server audit on a server instance for all databases */
SELECT TOP 1000 
	'[database_audit_specification_details]' AS TableName
	,[database_specification_id]
	,[audit_action_id]
	,[audit_action_name]
	,[class]
	,[class_desc]
	,[major_id]
	,[minor_id]
	,[audited_principal_id]
	,[audited_result]
	,[is_group]
FROM [master].[sys].[database_audit_specification_details]


(0 rows affected)

Total execution time: 00:00:00.010

TableName,database_specification_id,audit_action_id,audit_action_name,class,class_desc,major_id,minor_id,audited_principal_id,audited_result,is_group


In [26]:
/* Contains information about the database audit specifications in a SQL Server audit on a server instance. */
SELECT TOP 1000 
	'[database_audit_specifications]' AS TableName
	,[database_specification_id]
	,[name]
	,[create_date]
	,[modify_date]
	,[audit_guid]
	,[is_state_enabled]
FROM [master].[sys].[database_audit_specifications]


(0 rows affected)

Total execution time: 00:00:00.005

TableName,database_specification_id,name,create_date,modify_date,audit_guid,is_state_enabled


In [25]:
/* Returns a row for every audit action that can be reported in the audit log and every audit action group  */
--that can be configured as part of SQL Server Audit.  
SELECT TOP 1000 
	'[dm_audit_actions]' AS TableName
	,[action_id]
	,[name]
	,[class_desc]
	,[covering_action_name]
	,[parent_class_desc]
	,[covering_parent_action_name]
	,[configuration_level]
	,[containing_group_name]
	,[action_in_log]
FROM [master].[sys].[dm_audit_actions]


(616 rows affected)

Total execution time: 00:00:00.018

TableName,action_id,name,class_desc,covering_action_name,parent_class_desc,covering_parent_action_name,configuration_level,containing_group_name,action_in_log
[dm_audit_actions],R,REVOKE,DATABASE,DATABASE_PERMISSION_CHANGE_GROUP,SERVER,DATABASE_PERMISSION_CHANGE_GROUP,NULL,DATABASE_PERMISSION_CHANGE_GROUP,1
[dm_audit_actions],D,DENY,DATABASE,DATABASE_PERMISSION_CHANGE_GROUP,SERVER,DATABASE_PERMISSION_CHANGE_GROUP,NULL,DATABASE_PERMISSION_CHANGE_GROUP,1
[dm_audit_actions],G,GRANT,DATABASE,DATABASE_PERMISSION_CHANGE_GROUP,SERVER,DATABASE_PERMISSION_CHANGE_GROUP,NULL,DATABASE_PERMISSION_CHANGE_GROUP,1
[dm_audit_actions],GWG,GRANT WITH GRANT,DATABASE,DATABASE_PERMISSION_CHANGE_GROUP,SERVER,DATABASE_PERMISSION_CHANGE_GROUP,NULL,DATABASE_PERMISSION_CHANGE_GROUP,1
[dm_audit_actions],RWG,REVOKE WITH GRANT,DATABASE,DATABASE_PERMISSION_CHANGE_GROUP,SERVER,DATABASE_PERMISSION_CHANGE_GROUP,NULL,DATABASE_PERMISSION_CHANGE_GROUP,1
[dm_audit_actions],RWC,REVOKE WITH CASCADE,DATABASE,DATABASE_PERMISSION_CHANGE_GROUP,SERVER,DATABASE_PERMISSION_CHANGE_GROUP,NULL,DATABASE_PERMISSION_CHANGE_GROUP,1
[dm_audit_actions],DWC,DENY WITH CASCADE,DATABASE,DATABASE_PERMISSION_CHANGE_GROUP,SERVER,DATABASE_PERMISSION_CHANGE_GROUP,NULL,DATABASE_PERMISSION_CHANGE_GROUP,1
[dm_audit_actions],R,REVOKE,OBJECT,NULL,DATABASE,SCHEMA_OBJECT_PERMISSION_CHANGE_GROUP,NULL,SCHEMA_OBJECT_PERMISSION_CHANGE_GROUP,1
[dm_audit_actions],D,DENY,OBJECT,NULL,DATABASE,SCHEMA_OBJECT_PERMISSION_CHANGE_GROUP,NULL,SCHEMA_OBJECT_PERMISSION_CHANGE_GROUP,1
[dm_audit_actions],G,GRANT,OBJECT,NULL,DATABASE,SCHEMA_OBJECT_PERMISSION_CHANGE_GROUP,NULL,SCHEMA_OBJECT_PERMISSION_CHANGE_GROUP,1


In [24]:
/* Returns a table that maps the class_type field in the audit log to the class_desc field in sys.dm_audit_actions  */
SELECT TOP 1000 
	'[dm_audit_class_type_map]' AS TableName
	,[class_type]
	,[class_type_desc]
	,[securable_class_desc]
FROM [master].[sys].[dm_audit_class_type_map]


(105 rows affected)

Total execution time: 00:00:00.018

TableName,class_type,class_type_desc,securable_class_desc
[dm_audit_class_type_map],DB,DATABASE,DATABASE
[dm_audit_class_type_map],OB,OBJECT,OBJECT
[dm_audit_class_type_map],TY,TYPE,TYPE
[dm_audit_class_type_map],SC,SCHEMA,SCHEMA
[dm_audit_class_type_map],SX,XML SCHEMA COLLECTION,XML SCHEMA COLLECTION
[dm_audit_class_type_map],AS,ASSEMBLY,ASSEMBLY
[dm_audit_class_type_map],US,USER,USER
[dm_audit_class_type_map],RL,ROLE,ROLE
[dm_audit_class_type_map],AR,APPLICATION ROLE,APPLICATION ROLE
[dm_audit_class_type_map],MT,MESSAGE TYPE,MESSAGE TYPE


In [23]:
/* Returns a row for each server audit indicating the current state of the audit. */
SELECT TOP 1000 
	'[dm_server_audit_status]' AS TableName
	,[audit_id]
	,[name]
	,[status]
	,[status_desc]
	,[status_time]
	,[event_session_address]
	,[audit_file_path]
	,[audit_file_size]
FROM [master].[sys].[dm_server_audit_status]


(1 row affected)

Total execution time: 00:00:00.007

TableName,audit_id,name,status,status_desc,status_time,event_session_address,audit_file_path,audit_file_size
[dm_server_audit_status],65545,JobHistory_Audit,1,STARTED,2022-02-14 14:07:56.8520000,0x0000023D7A794FC1,C:\Audit\JobHistory_Audit_7FD5C90E-9F51-40F5-94A1-CB381A44E411_0_132893212768490000.sqlaudit,7680


In [22]:
/* Contains information about the server audit specification details (actions) in a SQL Server audit on a server instance.*/
SELECT TOP 1000 
	'[server_audit_specification_details]' AS TableName
	,[server_specification_id]
	,[audit_action_id]
	,[audit_action_name]
	,[class]
	,[class_desc]
	,[major_id]
	,[minor_id]
	,[audited_principal_id]
	,[audited_result]
	,[is_group]
FROM [master].[sys].[server_audit_specification_details]


(0 rows affected)

Total execution time: 00:00:00.005

TableName,server_specification_id,audit_action_id,audit_action_name,class,class_desc,major_id,minor_id,audited_principal_id,audited_result,is_group


In [21]:
/* Contains information about the server audit specifications in a SQL Server audit on a server instance */
SELECT TOP 1000 
	'server_audit_specifications' AS TableName
	,[server_specification_id]
	,[name]
	,[create_date]
	,[modify_date]
	,[audit_guid]
	,[is_state_enabled]
FROM [master].[sys].[server_audit_specifications]


(0 rows affected)

Total execution time: 00:00:00.004

TableName,server_specification_id,name,create_date,modify_date,audit_guid,is_state_enabled


In [20]:

/* Contains one row for each SQL Server audit in a server instance */
SELECT TOP 1000 
	'[server_audits]' AS TableName
	,[audit_id]
	,[name]
	,[audit_guid]
	,[create_date]
	,[modify_date]
	,[principal_id]
	,[type]
	,[type_desc]
	,[on_failure]
	,[on_failure_desc]
	,[is_state_enabled]
	,[queue_delay]
FROM [master].[sys].[server_audits]


(1 row affected)

Total execution time: 00:00:00.013

TableName,audit_id,name,audit_guid,create_date,modify_date,principal_id,type,type_desc,on_failure,on_failure_desc,is_state_enabled,queue_delay
[server_audits],65545,JobHistory_Audit,7fd5c90e-9f51-40f5-94a1-cb381a44e411,2022-02-14 14:07:56.843,2022-02-14 14:07:56.843,1,FL,FILE,0,CONTINUE,1,1000


In [19]:
/* Contains extended information about the file audit type in a SQL Server audit on a server instance */
SELECT TOP 1000 
	'[server_file_audits]' AS TableName
	,[audit_id]
	,[name]
	,[audit_guid]
	,[create_date]
	,[modify_date]
	,[principal_id]
	,[type]
	,[type_desc]
	,[on_failure]
	,[on_failure_desc]
	,[is_state_enabled]
	,[queue_delay]
	,[max_file_size]
	,[max_rollover_files]
	,[reserve_disk_space]
	,[log_file_path]
	,[log_file_name]
FROM [master].[sys].[server_file_audits]


(1 row affected)

Total execution time: 00:00:00.005

TableName,audit_id,name,audit_guid,create_date,modify_date,principal_id,type,type_desc,on_failure,on_failure_desc,is_state_enabled,queue_delay,max_file_size,max_rollover_files,reserve_disk_space,log_file_path,log_file_name
[server_file_audits],65545,JobHistory_Audit,7fd5c90e-9f51-40f5-94a1-cb381a44e411,2022-02-14 14:07:56.843,2022-02-14 14:07:56.843,1,FL,FILE,0,CONTINUE,1,1000,0,2147483647,0,C:\Audit\,JobHistory_Audit_7FD5C90E-9F51-40F5-94A1-CB381A44E411.sqlaudit


In [ ]:
/* Get Audit information for specific Audits */
SELECT AuditFile.*
FROM sys.dm_server_audit_status AS AuditStatus
CROSS APPLY sys.fn_get_audit_file(AuditStatus.audit_file_path, DEFAULT, DEFAULT) AS AuditFile



In [18]:
/* Disable and Drop Audit */
USE [master]
GO

DECLARE @Audit_Name sysname ='server_file_audits',
	    @sqlcmd VARCHAR(MAX)

IF EXISTS(SELECT name FROM sys.server_audits WHERE is_state_enabled = 1 AND Name =@Audit_Name)
BEGIN
	SET @sqlcmd ='ALTER SERVER AUDIT [' + @Audit_Name + '] WITH (STATE = OFF)'
	EXEC (@sqlcmd)

    SET @sqlcmd=  'DROP SERVER AUDIT ['+ @Audit_Name + ']'
    EXEC (@sqlcmd)
END


Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.006

In [13]:
/* Drop audit that exists and is disabled */
USE [master]
GO

DECLARE @Audit_Name sysname ='JobHistory_Audit',
	    @sqlcmd VARCHAR(MAX)

IF EXISTS(SELECT name FROM sys.server_audits WHERE is_state_enabled = 0 AND Name =@Audit_Name)
BEGIN
	SET @sqlcmd=  'DROP SERVER AUDIT ['+ @Audit_Name + ']'
    EXEC (@sqlcmd)
END


Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.007

In [ ]:

SELECT name FROM sys.server_audits WHERE is_state_enabled = 1 AND Name =$Audit_Name

In [14]:
/* 02 Audit Job changes.sql  */
--=====================================
--Create Job history Audits
--=====================================

USE [master]
GO


CREATE SERVER AUDIT [JobHistory_Audit] TO FILE (
	FILEPATH = N'C:\Audit'
	,MAXSIZE = 0 MB
	,MAX_ROLLOVER_FILES = 2147483647
	,RESERVE_DISK_SPACE = OFF
	)
	WITH (
			QUEUE_DELAY = 1000
			,ON_FAILURE = CONTINUE
			)
GO

USE [msdb]
GO



CREATE DATABASE AUDIT SPECIFICATION [DatabaseAuditSpecification-EnterDate]
FOR SERVER AUDIT [JobHistory_Audit] ADD (
		UPDATE ON OBJECT::[dbo].[sysjobs] BY [public]
		)
	,ADD (
		INSERT ON OBJECT::[dbo].[sysjobs] BY [public]
		)
	,ADD (
		DELETE ON OBJECT::[dbo].[sysjobs] BY [public]
		)
GO

ALTER DATABASE AUDIT SPECIFICATION [DatabaseAuditSpecification-EnterDate]
	WITH (STATE = ON)

USE master
GO

ALTER SERVER AUDIT [JobHistory_Audit]
	WITH (STATE = ON)
GO

SELECT statement
	,*
FROM fn_get_audit_file('C:\Audit\jobHistory*', DEFAULT, DEFAULT)
GO

--Sysjobsteps
USE MASTER
GO

CREATE SERVER AUDIT [JobStepsHistory_Audit] TO FILE (
	FILEPATH = N'C:\Audit'
	,MAXSIZE = 0 MB
	,MAX_ROLLOVER_FILES = 2147483647
	,RESERVE_DISK_SPACE = OFF
	)
	WITH (
			QUEUE_DELAY = 1000
			,ON_FAILURE = CONTINUE
			)
GO

USE MSDB
GO

CREATE DATABASE AUDIT SPECIFICATION [DatabaseAuditSpecification-JobSteps]
FOR SERVER AUDIT [JobStepsHistory_Audit] ADD (
		UPDATE ON OBJECT::[dbo].[sysjobsteps] BY [public]
		)
	,ADD (
		INSERT ON OBJECT::[dbo].[sysjobsteps] BY [public]
		)
	,ADD (
		DELETE ON OBJECT::[dbo].[sysjobsteps] BY [public]
		)

ALTER DATABASE AUDIT SPECIFICATION [DatabaseAuditSpecification-JobSteps]
	WITH (STATE = ON)

USE master
GO

ALTER SERVER AUDIT [JobStepsHistory_Audit]
	WITH (STATE = ON)
GO

SELECT statement
	,*
FROM fn_get_audit_file('C:\Audit\jobStepsHistory*', DEFAULT, DEFAULT)

SELECT statement
	,*
FROM fn_get_audit_file('C:\Audit\jobHistory*', DEFAULT, DEFAULT)


Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

: Msg 15530, Level 16, State 1, Line 27
The audit specification with name "DatabaseAuditSpecification-EnterDate" already exists.

Commands completed successfully.

...

Total execution time: 00:00:00.044

statement,event_time,sequence_number,action_id,succeeded,permission_bitmask,is_column_permission,session_id,server_principal_id,database_principal_id,target_server_principal_id,target_database_principal_id,object_id,class_type,session_server_principal_name,server_principal_name,server_principal_sid,database_principal_name,target_server_principal_name,target_server_principal_sid,target_database_principal_name,server_instance_name,database_name,schema_name,object_name,statement,additional_information,file_name,audit_file_offset,user_defined_event_id,user_defined_information,audit_schema_version,sequence_group_id,transaction_id,client_ip,application_name,duration_milliseconds,response_rows,affected_rows,connection_id,data_sensitivity_information,host_name
,2022-02-14 13:48:12.9286940,1,AUSC,1,0x00000000000000000000000000000000,0,63,259,0,0,0,0,A,MicrosoftAccount\martincroft22@gmail.com,JURGEN\marti,0x0105000000000005150000001403843556349AE9374930EAE9030000,,,NULL,,JURGEN,,,,,"<action_info xmlns=""http://schemas.microsoft.com/sqlserver/2008/sqlaudit_data""><session><![CDATA[JobHistory_Audit$A]]></session><action>event enabled</action><startup_type>manual</startup_type><object><![CDATA[audit_event]]></object></action_info>",C:\Audit\JobHistory_Audit_121CCCE8-5095-4C6D-84D9-30A01745785F_0_132893200929250000.sqlaudit,6144,0,,1,0x00000000000000000000000000000000,0,local machine,azdata-Query,0,0,0,8dbad2f1-6d0b-4234-906f-dbc463f22686,,JURGEN
,2022-02-14 13:51:27.7801185,1,AUSC,1,0x00000000000000000000000000000000,0,56,259,0,0,0,0,A,MicrosoftAccount\martincroft22@gmail.com,JURGEN\marti,0x0105000000000005150000001403843556349AE9374930EAE9030000,,,NULL,,JURGEN,,,,,"<action_info xmlns=""http://schemas.microsoft.com/sqlserver/2008/sqlaudit_data""><session><![CDATA[JobHistory_Audit$A]]></session><action>destroyed</action></action_info>",C:\Audit\JobHistory_Audit_121CCCE8-5095-4C6D-84D9-30A01745785F_0_132893200929250000.sqlaudit,7680,0,,1,0x00000000000000000000000000000000,0,local machine,Microsoft SQL Server Management Studio,0,0,0,e84701da-9349-455f-927f-8e5930d4f02e,,JURGEN
,2022-02-14 13:51:27.7801185,1,AUSC,1,0x00000000000000000000000000000000,0,56,259,0,0,0,0,A,MicrosoftAccount\martincroft22@gmail.com,JURGEN\marti,0x0105000000000005150000001403843556349AE9374930EAE9030000,,,NULL,,JURGEN,,,,,"<action_info xmlns=""http://schemas.microsoft.com/sqlserver/2008/sqlaudit_data""><session><![CDATA[JobHistory_Audit$A]]></session><action>event disabled</action><object><![CDATA[audit_event]]></object></action_info>",C:\Audit\JobHistory_Audit_121CCCE8-5095-4C6D-84D9-30A01745785F_0_132893200929250000.sqlaudit,8704,0,,1,0x00000000000000000000000000000000,0,local machine,Microsoft SQL Server Management Studio,0,0,0,e84701da-9349-455f-927f-8e5930d4f02e,,JURGEN
,2022-02-14 13:51:34.8265935,1,AUSC,1,0x00000000000000000000000000000000,0,56,259,0,0,0,0,A,MicrosoftAccount\martincroft22@gmail.com,JURGEN\marti,0x0105000000000005150000001403843556349AE9374930EAE9030000,,,NULL,,JURGEN,,,,,"<action_info xmlns=""http://schemas.microsoft.com/sqlserver/2008/sqlaudit_data""><session><![CDATA[JobHistory_Audit$A]]></session><action>event enabled</action><startup_type>manual</startup_type><object><![CDATA[audit_event]]></object></action_info>",C:\Audit\JobHistory_Audit_121CCCE8-5095-4C6D-84D9-30A01745785F_0_132893202948250000.sqlaudit,6144,0,,1,0x00000000000000000000000000000000,0,local machine,Microsoft SQL Server Management Studio,0,0,0,011982d3-74cf-474a-adc3-3cc960a32141,,JURGEN
,2022-02-14 13:57:47.3637706,1,AUSC,1,0x00000000000000000000000000000000,0,59,259,0,0,0,0,A,MicrosoftAccount\martincroft22@gmail.com,JURGEN\marti,0x0105000000000005150000001403843556349AE9374930EAE9030000,,,NULL,,JURGEN,,,,,"<action_info xmlns=""http://schemas.microsoft.com/sqlserver/2008/sqlaudit_data""><session><![CDATA[JobHistory_Audit$A]]></session><action>destroyed</action></action_info>",C:\Audit\JobHistory_Audit_121CCCE8-5095-4C6D-84D9-30A01745785F_0_132893202948250000.sqlaudit,7680,0

: Msg 15530, Level 16, State 1, Line 73
The audit specification with name "DatabaseAuditSpecification-JobSteps" already exists.

statement,event_time,sequence_number,action_id,succeeded,permission_bitmask,is_column_permission,session_id,server_principal_id,database_principal_id,target_server_principal_id,target_database_principal_id,object_id,class_type,session_server_principal_name,server_principal_name,server_principal_sid,database_principal_name,target_server_principal_name,target_server_principal_sid,target_database_principal_name,server_instance_name,database_name,schema_name,object_name,statement,additional_information,file_name,audit_file_offset,user_defined_event_id,user_defined_information,audit_schema_version,sequence_group_id,transaction_id,client_ip,application_name,duration_milliseconds,response_rows,affected_rows,connection_id,data_sensitivity_information,host_name
,2022-02-14 13:48:12.9546246,1,AUSC,1,0x00000000000000000000000000000000,0,63,259,0,0,0,0,A,MicrosoftAccount\martincroft22@gmail.com,JURGEN\marti,0x0105000000000005150000001403843556349AE9374930EAE9030000,,,NULL,,JURGEN,,,,,"<action_info xmlns=""http://schemas.microsoft.com/sqlserver/2008/sqlaudit_data""><session><![CDATA[JobStepsHistory_Audit$A]]></session><action>event enabled</action><startup_type>manual</startup_type><object><![CDATA[audit_event]]></object></action_info>",C:\Audit\JobStepsHistory_Audit_C77014A1-4780-46F9-A723-132B4557D6F8_0_132893200929530000.sqlaudit,6144,0,,1,0x00000000000000000000000000000000,0,local machine,azdata-Query,0,0,0,8dbad2f1-6d0b-4234-906f-dbc463f22686,,JURGEN
,2022-02-14 13:58:20.0693250,1,AUSC,1,0x00000000000000000000000000000000,0,59,259,0,0,0,0,A,MicrosoftAccount\martincroft22@gmail.com,JURGEN\marti,0x0105000000000005150000001403843556349AE9374930EAE9030000,,,NULL,,JURGEN,,,,,"<action_info xmlns=""http://schemas.microsoft.com/sqlserver/2008/sqlaudit_data""><session><![CDATA[JobStepsHistory_Audit$A]]></session><action>destroyed</action></action_info>",C:\Audit\JobStepsHistory_Audit_C77014A1-4780-46F9-A723-132B4557D6F8_0_132893200929530000.sqlaudit,7680,0,,1,0x00000000000000000000000000000000,0,local machine,Microsoft SQL Server Management Studio - Query,0,0,0,aa3ac364-46a7-4a97-8d7f-9dc2514d4156,,JURGEN
,2022-02-14 13:58:20.0703222,1,AUSC,1,0x00000000000000000000000000000000,0,59,259,0,0,0,0,A,MicrosoftAccount\martincroft22@gmail.com,JURGEN\marti,0x0105000000000005150000001403843556349AE9374930EAE9030000,,,NULL,,JURGEN,,,,,"<action_info xmlns=""http://schemas.microsoft.com/sqlserver/2008/sqlaudit_data""><session><![CDATA[JobStepsHistory_Audit$A]]></session><action>event disabled</action><object><![CDATA[audit_event]]></object></action_info>",C:\Audit\JobStepsHistory_Audit_C77014A1-4780-46F9-A723-132B4557D6F8_0_132893200929530000.sqlaudit,8704,0,,1,0x00000000000000000000000000000000,0,local machine,Microsoft SQL Server Management Studio - Query,0,0,0,aa3ac364-46a7-4a97-8d7f-9dc2514d4156,,JURGEN
,2022-02-14 14:04:54.4869523,1,AUSC,1,0x00000000000000000000000000000000,0,63,259,0,0,0,0,A,MicrosoftAccount\martincroft22@gmail.com,JURGEN\marti,0x0105000000000005150000001403843556349AE9374930EAE9030000,,,NULL,,JURGEN,,,,,"<action_info xmlns=""http://schemas.microsoft.com/sqlserver/2008/sqlaudit_data""><session><![CDATA[JobStepsHistory_Audit$A]]></session><action>event enabled</action><startup_type>manual</startup_type><object><![CDATA[audit_event]]></object></action_info>",C:\Audit\JobStepsHistory_Audit_C77014A1-4780-46F9-A723-132B4557D6F8_0_132893210944840000.sqlaudit,6144,0,,1,0x00000000000000000000000000000000,0,local machine,azdata-Query,0,0,0,8dbad2f1-6d0b-4234-906f-dbc463f22686,,JURGEN
,2022-02-14 14:05:20.2921365,1,AUSC,1,0x00000000000000000000000000000000,0,63,259,0,0,0,0,A,MicrosoftAccount\martincroft22@gmail.com,JURGEN\marti,0x0105000000000005150000001403843556349AE9374930EAE9030000,,,NULL,,JURGEN,,,,,"<action_info xmlns=""http://schemas.microsoft.com/sqlserver/2008/sqlaudit_data""><session><![CDATA[JobStepsHistory_Audit$A]]></session><action>destroyed</action></action_info>",C:\Audit\JobStepsHistory_Audit_C77014A1-4780-46F9-A723-132B4557D

statement,event_time,sequence_number,action_id,succeeded,permission_bitmask,is_column_permission,session_id,server_principal_id,database_principal_id,target_server_principal_id,target_database_principal_id,object_id,class_type,session_server_principal_name,server_principal_name,server_principal_sid,database_principal_name,target_server_principal_name,target_server_principal_sid,target_database_principal_name,server_instance_name,database_name,schema_name,object_name,statement,additional_information,file_name,audit_file_offset,user_defined_event_id,user_defined_information,audit_schema_version,sequence_group_id,transaction_id,client_ip,application_name,duration_milliseconds,response_rows,affected_rows,connection_id,data_sensitivity_information,host_name
,2022-02-14 13:48:12.9286940,1,AUSC,1,0x00000000000000000000000000000000,0,63,259,0,0,0,0,A,MicrosoftAccount\martincroft22@gmail.com,JURGEN\marti,0x0105000000000005150000001403843556349AE9374930EAE9030000,,,NULL,,JURGEN,,,,,"<action_info xmlns=""http://schemas.microsoft.com/sqlserver/2008/sqlaudit_data""><session><![CDATA[JobHistory_Audit$A]]></session><action>event enabled</action><startup_type>manual</startup_type><object><![CDATA[audit_event]]></object></action_info>",C:\Audit\JobHistory_Audit_121CCCE8-5095-4C6D-84D9-30A01745785F_0_132893200929250000.sqlaudit,6144,0,,1,0x00000000000000000000000000000000,0,local machine,azdata-Query,0,0,0,8dbad2f1-6d0b-4234-906f-dbc463f22686,,JURGEN
,2022-02-14 13:51:27.7801185,1,AUSC,1,0x00000000000000000000000000000000,0,56,259,0,0,0,0,A,MicrosoftAccount\martincroft22@gmail.com,JURGEN\marti,0x0105000000000005150000001403843556349AE9374930EAE9030000,,,NULL,,JURGEN,,,,,"<action_info xmlns=""http://schemas.microsoft.com/sqlserver/2008/sqlaudit_data""><session><![CDATA[JobHistory_Audit$A]]></session><action>destroyed</action></action_info>",C:\Audit\JobHistory_Audit_121CCCE8-5095-4C6D-84D9-30A01745785F_0_132893200929250000.sqlaudit,7680,0,,1,0x00000000000000000000000000000000,0,local machine,Microsoft SQL Server Management Studio,0,0,0,e84701da-9349-455f-927f-8e5930d4f02e,,JURGEN
,2022-02-14 13:51:27.7801185,1,AUSC,1,0x00000000000000000000000000000000,0,56,259,0,0,0,0,A,MicrosoftAccount\martincroft22@gmail.com,JURGEN\marti,0x0105000000000005150000001403843556349AE9374930EAE9030000,,,NULL,,JURGEN,,,,,"<action_info xmlns=""http://schemas.microsoft.com/sqlserver/2008/sqlaudit_data""><session><![CDATA[JobHistory_Audit$A]]></session><action>event disabled</action><object><![CDATA[audit_event]]></object></action_info>",C:\Audit\JobHistory_Audit_121CCCE8-5095-4C6D-84D9-30A01745785F_0_132893200929250000.sqlaudit,8704,0,,1,0x00000000000000000000000000000000,0,local machine,Microsoft SQL Server Management Studio,0,0,0,e84701da-9349-455f-927f-8e5930d4f02e,,JURGEN
,2022-02-14 13:51:34.8265935,1,AUSC,1,0x00000000000000000000000000000000,0,56,259,0,0,0,0,A,MicrosoftAccount\martincroft22@gmail.com,JURGEN\marti,0x0105000000000005150000001403843556349AE9374930EAE9030000,,,NULL,,JURGEN,,,,,"<action_info xmlns=""http://schemas.microsoft.com/sqlserver/2008/sqlaudit_data""><session><![CDATA[JobHistory_Audit$A]]></session><action>event enabled</action><startup_type>manual</startup_type><object><![CDATA[audit_event]]></object></action_info>",C:\Audit\JobHistory_Audit_121CCCE8-5095-4C6D-84D9-30A01745785F_0_132893202948250000.sqlaudit,6144,0,,1,0x00000000000000000000000000000000,0,local machine,Microsoft SQL Server Management Studio,0,0,0,011982d3-74cf-474a-adc3-3cc960a32141,,JURGEN
,2022-02-14 13:57:47.3637706,1,AUSC,1,0x00000000000000000000000000000000,0,59,259,0,0,0,0,A,MicrosoftAccount\martincroft22@gmail.com,JURGEN\marti,0x0105000000000005150000001403843556349AE9374930EAE9030000,,,NULL,,JURGEN,,,,,"<action_info xmlns=""http://schemas.microsoft.com/sqlserver/2008/sqlaudit_data""><session><![CDATA[JobHistory_Audit$A]]></session><action>destroyed</action></action_info>",C:\Audit\JobHistory_Audit_121CCCE8-5095-4C6D-84D9-30A01745785F_0_132893202948250000.sqlaudit,7680,0

## [Home](file:///C:/SQLScriptsLibrary/Home.ipynb)